In [3]:
tickers = [ "BEEF3.SA", "PETR4.SA", "SOJA3.SA", "GGBR3.SA", "CSNA3.SA", "VALE3.SA", "JBSS3.SA", "BRFS3.SA", "SUZB3.SA"]

In [6]:
import os
import pandas as pd

# Diretório base dos experimentos
base_dir = r"c:\Users\leona\pyhtonscripts\ExperimentosTeste"

# Função para processar os melhores resultados e capturar hiperparâmetros
def processar_experimentos(base_dir):
    resultados = []

    # Lista de experimentos
    experimentos = ["Macro", "Micro", "Kfold", "MacroMicro"]

    for experimento in experimentos:
        exp_dir = os.path.join(base_dir, experimento)
        if not os.path.exists(exp_dir):
            print(f"Diretório não encontrado para o experimento: {experimento}")
            continue

        # Iterar sobre os tickers
        for ticker in os.listdir(exp_dir):
            ticker_dir = os.path.join(exp_dir, ticker)
            if not os.path.isdir(ticker_dir):
                continue

            # Iterar sobre as janelas
            for janela in os.listdir(ticker_dir):
                janela_dir = os.path.join(ticker_dir, janela)
                if not os.path.isdir(janela_dir):
                    continue

                # Arquivos de métricas e hiperparâmetros
                metrics_file = os.path.join(janela_dir, f"{ticker}_{janela}_metrics.csv")
                hyperparams_file = os.path.join(janela_dir, f"{ticker}_{janela}_hiperparametros.csv")

                if os.path.exists(metrics_file) and os.path.exists(hyperparams_file):
                    # Carregar métricas
                    df_metrics = pd.read_csv(metrics_file)
                    
                    # Identificar a métrica de R2 correta
                    r2_col = None
                    if "Teste R2" in df_metrics.columns:
                        r2_col = "Teste R2"
                    elif "CV Avg R2" in df_metrics.columns:
                        r2_col = "CV Avg R2"
                    else:
                        print(f"Coluna de R2 não encontrada no arquivo: {metrics_file}")
                        continue
                    
                    # Identificar a melhor métrica (maior R2)
                    melhor_resultado = df_metrics.loc[df_metrics[r2_col].idxmax()]

                    # Carregar hiperparâmetros
                    df_hyperparams = pd.read_csv(hyperparams_file)

                    # Adicionar hiperparâmetros ao resultado
                    for col in df_hyperparams.columns:
                        melhor_resultado[col] = df_hyperparams[col].iloc[0]

                    # Adicionar informações do experimento e janela
                    melhor_resultado['Experimento'] = experimento
                    melhor_resultado['Janela'] = janela

                    # Adicionar aos resultados
                    resultados.append(melhor_resultado)

    # Consolidar resultados em um único DataFrame
    if resultados:
        df_resultados = pd.DataFrame(resultados)
        return df_resultados
    else:
        print("Nenhum resultado encontrado.")
        return None

# Função para salvar a tabela consolidada
def salvar_tabela_consolidada(df_resultados, output_path):
    if df_resultados is None or df_resultados.empty:
        print("Nenhum dado disponível para salvar.")
        return

    # Salvar tabela consolidada
    df_resultados.to_csv(output_path, index=False)
    print(f"Tabela consolidada salva em: {output_path}")

# Processar os experimentos
df_resultados = processar_experimentos(base_dir)

# Salvar a tabela consolidada
output_file = os.path.join(base_dir, "tabela_consolidada_com_hiperparametros.csv")
salvar_tabela_consolidada(df_resultados, output_file)

C:\Users\leona\AppData\Local\Temp\ipykernel_25396\3646470727.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melhor_resultado[col] = df_hyperparams[col].iloc[0]
C:\Users\leona\AppData\Local\Temp\ipykernel_25396\3646470727.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melhor_resultado[col] = df_hyperparams[col].iloc[0]
C:\Users\leona\AppData\Local\Temp\ipykernel_25396\3646470727.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

Tabela consolidada salva em: c:\Users\leona\pyhtonscripts\ExperimentosTeste\tabela_consolidada_com_hiperparametros.csv


C:\Users\leona\AppData\Local\Temp\ipykernel_25396\3646470727.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melhor_resultado[col] = df_hyperparams[col].iloc[0]
C:\Users\leona\AppData\Local\Temp\ipykernel_25396\3646470727.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melhor_resultado[col] = df_hyperparams[col].iloc[0]
C:\Users\leona\AppData\Local\Temp\ipykernel_25396\3646470727.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

In [10]:
import pandas as pd
import os
import numpy as np
import warnings
import seaborn as sns # Para heatmaps
import matplotlib.pyplot as plt # Para exibir e salvar plots

# Ignore warnings for cleaner output (optional)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=UserWarning) # Ignorar warnings do matplotlib/seaborn


# --- Configuration ---
# Adjust these paths based on your directory structure
macro_base_path = r'c:\Users\leona\pyhtonscripts\CodigoExperimentos\ExperimentoFeatures\dados_macro_csv_processados_tratados'
micro_base_path = r'c:\Users\leona\pyhtonscripts\CodigoExperimentos\ExperimentoFeatures\dataframesMicro'
output_base_path = r'c:\Users\leona\pyhtonscripts\CodigoExperimentos\ExperimentoFeatures\CorrelationOutputs' # Pasta para salvar CSVs e Plots

# Create output directory if it doesn't exist
os.makedirs(output_base_path, exist_ok=True)

# List of tickers to process (using the list from the first cell of the notebook)
# Removido PETR4.SA temporariamente se não tiver dados ou para teste
tickers = [ "BEEF3.SA", "SOJA3.SA", "GGBR3.SA", "CSNA3.SA", "VALE3.SA", "JBSS3.SA", "BRFS3.SA", "SUZB3.SA"]

# Potential names for the date column (case-insensitive check)
possible_date_cols = ['Data', 'Date']
# Potential names for the target stock price column in MICRO data (case-insensitive check)
possible_micro_target_cols = ['Close', 'Adj Close', 'Preço']
# Potential names for the target stock price column in MACRO data (case-insensitive check)
possible_macro_target_cols = ['Close_Target'] # Based on BEEF3.SA_MacroData_Tratado.csv example
# Macroeconomic columns to correlate with (from the macro dataset)
macro_corr_cols = ['TaxaCambio', 'Selic', 'PIB', 'IPCA']
# --- End Configuration ---

def find_column(df_columns, possible_names):
    """Helper function to find the first matching column name, case-insensitive."""
    df_columns_lower = [col.lower() for col in df_columns]
    for name in possible_names:
        try:
            idx = df_columns_lower.index(name.lower())
            return df_columns[idx] # Return the original case column name
        except ValueError:
            continue
    return None

# List to store correlation results for the final summary table
# Modificado para estrutura mais genérica
final_summary_correlations = []

# --- Processing Loop ---
for ticker in tickers:
    print("-" * 30)
    print(f"Processing ticker: {ticker}")

    # Construct file paths
    macro_file_name = f"{ticker}_MacroData_Tratado.csv"
    macro_file_path = os.path.join(macro_base_path, macro_file_name)
    micro_file_name = f"{ticker}_dados_unificados.csv"
    micro_file_path = os.path.join(micro_base_path, micro_file_name)

    # Paths for output files for this ticker
    ticker_output_csv = os.path.join(output_base_path, f"{ticker}_correlations.csv")
    ticker_output_plot_macro = os.path.join(output_base_path, f"{ticker}_macro_correlation_heatmap.png")
    ticker_output_plot_micro = os.path.join(output_base_path, f"{ticker}_micro_correlation_heatmap.png")


    macro_df = None
    micro_df = None
    macro_date_col = None
    macro_target_col = None
    micro_target_col = None
    can_correlate_macro = True
    can_correlate_micro = True

    # Lists to store correlations for the individual ticker CSV
    ticker_macro_corr_results = []
    ticker_micro_corr_results = []

    # --- Load Macro Data ---
    try:
        print(f"  Loading macro data: {macro_file_path}")
        macro_df_raw = pd.read_csv(macro_file_path)
        macro_date_col = find_column(macro_df_raw.columns, possible_date_cols)
        macro_target_col = find_column(macro_df_raw.columns, possible_macro_target_cols)

        if not macro_date_col:
            print(f"  Warning: No suitable date column ({'/'.join(possible_date_cols)}) found in {macro_file_name}. Cannot use macro data for time-series correlation.")
            can_correlate_macro = False
        else:
            print(f"  Found macro date column: '{macro_date_col}'")

        if not macro_target_col:
             print(f"  Warning: Macro target column ({'/'.join(possible_macro_target_cols)}) not found in {macro_file_name}. Cannot use macro data for correlation.")
             can_correlate_macro = False
        else:
             print(f"  Found macro target column: '{macro_target_col}'")

        if can_correlate_macro:
            macro_df = pd.read_csv(
                macro_file_path,
                parse_dates=[macro_date_col],
                index_col=macro_date_col
            )
            cols_to_check = macro_corr_cols + [macro_target_col]
            missing_cols = []
            for col in cols_to_check:
                if col in macro_df.columns:
                    macro_df[col] = pd.to_numeric(macro_df[col], errors='coerce')
                else:
                    if col in macro_corr_cols or col == macro_target_col:
                        missing_cols.append(col)

            if missing_cols:
                 print(f"  Warning: Essential macro columns missing or not found: {', '.join(missing_cols)}. Skipping macro correlation.")
                 can_correlate_macro = False
                 macro_df = None
            else:
                 macro_df.dropna(subset=cols_to_check, inplace=True)
                 if macro_df.empty:
                     print(f"  Warning: Macro data became empty after dropping NaN rows in essential columns. Skipping macro correlation.")
                     can_correlate_macro = False
                     macro_df = None
                 else:
                    print(f"  Processed Macro data for {ticker}.")

    except FileNotFoundError:
        print(f"  Error: Macro file not found: {macro_file_path}")
        can_correlate_macro = False
    except Exception as e:
        print(f"  Error processing macro file {macro_file_path}: {e}")
        can_correlate_macro = False

    # --- Load Micro Data ---
    try:
        print(f"  Loading micro data: {micro_file_path}")
        try:
            micro_df = pd.read_csv(
                micro_file_path,
                index_col=0,
                parse_dates=True
            )
            if not isinstance(micro_df.index, pd.DatetimeIndex):
                 raise ValueError("Index not DatetimeIndex")
            print(f"  Micro data loaded with date index from first column.")

        except (ValueError, TypeError, IndexError, FileNotFoundError): # Adicionado FileNotFoundError aqui
             if not os.path.exists(micro_file_path):
                  print(f"  Error: Micro file not found: {micro_file_path}")
                  can_correlate_micro = False
                  micro_df = None
             else:
                print("  Warning: Index parsing from first column failed or file not found initially. Attempting named date column.")
                try:
                    micro_df_raw = pd.read_csv(micro_file_path)
                    micro_date_col_named = find_column(micro_df_raw.columns, possible_date_cols)
                    if micro_date_col_named:
                        print(f"  Found micro date column: '{micro_date_col_named}'")
                        micro_df = pd.read_csv(
                            micro_file_path,
                            parse_dates=[micro_date_col_named],
                            index_col=micro_date_col_named
                        )
                    else:
                        print(f"  Warning: No suitable date column ({'/'.join(possible_date_cols)}) or parsable date index found in {micro_file_name}.")
                        can_correlate_micro = False
                        micro_df = None
                except Exception as fallback_e:
                     print(f"  Error during fallback micro loading for {micro_file_path}: {fallback_e}")
                     can_correlate_micro = False
                     micro_df = None


        if micro_df is not None:
            micro_target_col = find_column(micro_df.columns, possible_micro_target_cols)

            if not micro_target_col:
                print(f"  Warning: Micro target column ({'/'.join(possible_micro_target_cols)}) not found in {micro_file_name}. Skipping micro correlation.")
                can_correlate_micro = False
                micro_df = None
            else:
                print(f"  Found micro target column: '{micro_target_col}'")
                # Ensure all potential numeric columns are treated
                for col in micro_df.select_dtypes(include=np.number).columns:
                     micro_df[col] = pd.to_numeric(micro_df[col], errors='coerce')

                # Drop rows where target is NaN, check if empty
                micro_df = micro_df.dropna(subset=[micro_target_col])
                if micro_df.empty:
                    print(f"  Warning: No numeric data found in micro target column '{micro_target_col}' after cleaning.")
                    can_correlate_micro = False
                    micro_df = None
                else:
                    print(f"  Processed Micro data for {ticker}.")
        else:
             # If micro_df is None after loading attempts
             can_correlate_micro = False


    except Exception as e:
        # Catch-all for any unexpected error during micro loading
        print(f"  Unexpected error processing micro file {micro_file_path}: {e}")
        can_correlate_micro = False
        micro_df = None


    # --- Calculate & Visualize Macro Correlation (Macro Vars vs Micro Target) ---
    if can_correlate_macro and can_correlate_micro and macro_df is not None and micro_df is not None:
        print(f"\n  Calculating Macro correlations for {ticker} (Macro Vars vs Micro Target '{micro_target_col}'):")
        try:
            # Align dataframes
            merged_df_macro = macro_df[macro_corr_cols + [macro_target_col]].join(micro_df[[micro_target_col]], how='inner')

            if merged_df_macro.empty:
                print("  Warning: No overlapping dates found for macro correlation.")
            else:
                # Columns for this specific correlation
                cols_for_macro_corr = macro_corr_cols + [micro_target_col]
                merged_df_macro.dropna(subset=cols_for_macro_corr, inplace=True)

                if not merged_df_macro.empty and len(merged_df_macro) > 1: # Need at least 2 rows for correlation
                    # Calculate correlation matrix
                    correlation_matrix_macro = merged_df_macro[cols_for_macro_corr].corr(method='pearson')

                    # Extract correlation of micro target column with macro columns
                    macro_target_correlations = correlation_matrix_macro[micro_target_col][macro_corr_cols].dropna()

                    if not macro_target_correlations.empty:
                        print("  Correlations with Macro Variables:")
                        print(macro_target_correlations)

                        # Store for individual CSV
                        for feature, corr_value in macro_target_correlations.items():
                            ticker_macro_corr_results.append({
                                'Correlation_Type': 'Macro vs Micro Target',
                                'Variable_1': feature,
                                'Variable_2': micro_target_col,
                                'Correlation': corr_value
                            })
                            # Store for final summary table (MODIFICADO)
                            final_summary_correlations.append({
                                 'Ticker': ticker,
                                 'Feature_Type': 'Macro', # Nova coluna
                                 'Feature_Name': feature, # Nome genérico da feature
                                 'Correlation_Value': corr_value # Nome genérico do valor
                             })

                        # --- Visualize Macro Correlation ---
                        plt.figure(figsize=(8, 6))
                        sns.heatmap(correlation_matrix_macro[[micro_target_col]].reindex(macro_corr_cols), # Heatmap of Macro vs Micro Target
                                    annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5, vmin=-1, vmax=1)
                        plt.title(f'{ticker}: Macro Variable Correlation with {micro_target_col}')
                        plt.tight_layout()
                        plt.savefig(ticker_output_plot_macro)
                        plt.close() # Close the plot to free memory
                        print(f"  Saved macro correlation heatmap to: {ticker_output_plot_macro}")

                    else:
                        print("  Could not calculate valid macro correlations.")
                else:
                    print("  Warning: Not enough valid data points after merging/dropping NaNs for macro correlation.")

        except Exception as corr_e:
            print(f"  Error during macro correlation calculation/visualization: {corr_e}")
    else:
        print(f"\n  Skipping Macro correlation calculation for {ticker} due to data loading issues.")


    # --- Calculate & Visualize Micro Correlation (Internal) ---
    if can_correlate_micro and micro_df is not None:
        print(f"\n  Calculating Micro correlations for {ticker} (Internal):")
        try:
            # Select only numeric columns from micro_df
            micro_numeric_df = micro_df.select_dtypes(include=np.number)

            # Drop columns with no variance or too many NaNs before calculating correlation
            micro_numeric_df = micro_numeric_df.dropna(axis=1, how='all') # Drop cols that are all NaN
            micro_numeric_df = micro_numeric_df.loc[:, micro_numeric_df.apply(pd.Series.nunique) > 1] # Drop cols with only 1 unique value

            if micro_numeric_df.empty or micro_numeric_df.shape[1] < 2:
                 print("  Warning: Not enough numeric columns with variance for micro correlation.")
            elif micro_target_col not in micro_numeric_df.columns:
                 print(f"  Warning: Micro target column '{micro_target_col}' not found in numeric columns after cleaning.")
            else:
                # Calculate full micro correlation matrix
                correlation_matrix_micro = micro_numeric_df.corr(method='pearson')

                # Extract correlations of the target column with other micro columns
                micro_target_correlations = correlation_matrix_micro[micro_target_col].drop(micro_target_col, errors='ignore').dropna()

                if not micro_target_correlations.empty:
                     print(f"  Correlations with Micro Target ('{micro_target_col}'):")
                     print(micro_target_correlations)
                     # Store for individual CSV
                     for feature, corr_value in micro_target_correlations.items():
                         ticker_micro_corr_results.append({
                             'Correlation_Type': 'Micro vs Micro Target',
                             'Variable_1': feature,
                             'Variable_2': micro_target_col,
                             'Correlation': corr_value
                         })
                         # Store for final summary table (ADICIONADO)
                         final_summary_correlations.append({
                             'Ticker': ticker,
                             'Feature_Type': 'Micro', # Nova coluna
                             'Feature_Name': feature, # Nome genérico da feature
                             'Correlation_Value': corr_value # Nome genérico do valor
                         })

                # --- Visualize Micro Correlation ---
                # Check if correlation matrix is not empty before plotting
                if not correlation_matrix_micro.empty:
                    plt.figure(figsize=(10, 8)) # Ajuste o tamanho conforme necessário
                    sns.heatmap(correlation_matrix_micro, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5, vmin=-1, vmax=1)
                    plt.title(f'{ticker}: Internal Micro Variable Correlations')
                    plt.xticks(rotation=45, ha='right')
                    plt.yticks(rotation=0)
                    plt.tight_layout()
                    plt.savefig(ticker_output_plot_micro)
                    plt.close() # Close the plot to free memory
                    print(f"  Saved micro correlation heatmap to: {ticker_output_plot_micro}")
                else:
                    print("  Skipping micro heatmap generation as correlation matrix is empty.")


        except Exception as corr_e:
            print(f"  Error during micro correlation calculation/visualization: {corr_e}")

    else:
         print(f"\n  Skipping Micro correlation calculation for {ticker} due to data loading issues.")


    # --- Combine and Save Individual Ticker CSV ---
    if ticker_macro_corr_results or ticker_micro_corr_results:
        try:
            df_ticker_macro = pd.DataFrame(ticker_macro_corr_results)
            df_ticker_micro = pd.DataFrame(ticker_micro_corr_results)
            df_ticker_combined = pd.concat([df_ticker_macro, df_ticker_micro], ignore_index=True)

            df_ticker_combined.to_csv(ticker_output_csv, index=False)
            print(f"\n  Saved combined correlations for {ticker} to: {ticker_output_csv}")
        except Exception as save_e:
            print(f"\n  Error saving combined correlations CSV for {ticker}: {save_e}")
    else:
        print(f"\n  No correlation results generated for {ticker} to save.")


# --- Create Final Comparison Table ---
if not final_summary_correlations:
    print("\nNo correlation results were generated for the final summary table.")
else:
    print("\nGenerating final comparison table...")
    try:
        # Create DataFrame from results collected during the loop
        comparison_df = pd.DataFrame(final_summary_correlations)

        # Pivot the table (MODIFICADO)
        comparison_pivot = comparison_df.pivot_table(
            index=['Feature_Type', 'Feature_Name'], # MultiIndex para linhas
            columns='Ticker',
            values='Correlation_Value' # Usar a coluna de valor genérica
        )

        # Fill NaN values with 0 (ADICIONADO)
        comparison_pivot = comparison_pivot.fillna(0)

        # Ordenar para agrupar Macro e Micro
        comparison_pivot = comparison_pivot.sort_index(level=['Feature_Type', 'Feature_Name'])

        # Display the comparison table (unstyled first)
        print("\n--- Final Correlation Comparison Table (All Features vs. Micro Target) ---") # Título atualizado
        print(comparison_pivot)

        # --- Style the Final Table ---
        print("\n--- Styled Comparison Table ---")
        # Aplicar estilo
        styled_table = comparison_pivot.style.background_gradient(
            cmap='coolwarm', axis=None, vmin=-1, vmax=1
        ).format("{:.2f}") # Aplicar formatação após o fillna

        # Display styled table (works well in Jupyter)
        from IPython.display import display
        display(styled_table)

        # Optional: Save the unstyled pivot table to CSV (MODIFICADO)
        final_csv_output_path = os.path.join(output_base_path, "final_all_features_correlation_comparison.csv") # Nome do arquivo atualizado
        try:
            # Salvar a tabela com NaNs preenchidos como 0
            comparison_pivot.reset_index().to_csv(final_csv_output_path, index=False, float_format='%.4f')
            print(f"\nFinal comparison table saved to CSV: {final_csv_output_path}")
        except Exception as final_csv_save_e:
            print(f"\nError saving final comparison table to CSV: {final_csv_save_e}")

    except Exception as pivot_e:
        print(f"\nError creating or styling the final pivot table: {pivot_e}")
        print("\nRaw correlation results for summary:")
        print(pd.DataFrame(final_summary_correlations)) # Print raw data if pivot fails

print("-" * 30)
print("Processing finished.")

------------------------------
Processing ticker: BEEF3.SA
  Loading macro data: c:\Users\leona\pyhtonscripts\CodigoExperimentos\ExperimentoFeatures\dados_macro_csv_processados_tratados\BEEF3.SA_MacroData_Tratado.csv
  Found macro date column: 'Data'
  Found macro target column: 'Close_Target'
  Processed Macro data for BEEF3.SA.
  Loading micro data: c:\Users\leona\pyhtonscripts\CodigoExperimentos\ExperimentoFeatures\dataframesMicro\BEEF3.SA_dados_unificados.csv
  Micro data loaded with date index from first column.
  Found micro target column: 'Preço'
  Processed Micro data for BEEF3.SA.

  Calculating Macro correlations for BEEF3.SA (Macro Vars vs Micro Target 'Preço'):
  Correlations with Macro Variables:
TaxaCambio   -0.116434
Selic         0.500537
PIB           0.269073
IPCA         -0.266527
Name: Preço, dtype: float64
  Saved macro correlation heatmap to: c:\Users\leona\pyhtonscripts\CodigoExperimentos\ExperimentoFeatures\CorrelationOutputs\BEEF3.SA_macro_correlation_heatmap.p


Final comparison table saved to CSV: c:\Users\leona\pyhtonscripts\CodigoExperimentos\ExperimentoFeatures\CorrelationOutputs\final_all_features_correlation_comparison.csv
------------------------------
Processing finished.
